# Godbreaker
The [Godbreaker](https://2e.aonprd.com/Feats.aspx?ID=6049) feat (Monk 20, Wrestler 20) is the ultimate Super Sayan move - but how likely are you to pull it off completely?

Let's compare these builds:

1. Monk (**2.** Crushing Grab, **6.** Whilrling Throw, **19.** Perfected Form, **20.** Godbreaker)
2. Fighter (**2.** Wrestler Dedication, **4.** Crushing Grab, **8.** Whirling Throw, **10.** Agile Grace, **20.** Godbreaker)
3. Ranger (**1.** Flurry Edge, **2.** Wrestler Dedication, **4.** Crushing Grab, **8.** Whirling Throw, **20.** Godbreaker)
4. Any other martial (**2.** Wrestler Dedication, **4.** Crushing Grab, **8.** Whirling Throw, **20.** Godbreaker)
5. Any spellcaster (**2.** Wrestler Dedication, **4.** Crushing Grab, **8.** Whirling Throw, **20.** Godbreaker)

Assume that:
- All use +3 Greater Fearsome Keen Handwraps of Mighty Blows
- All martials have an Apex item for +1 STR; start at +4 STR and increase it at all bumps
- Non-martials start at +3 STR and increase it at all bumps
- All perform the second and third strike with an agile unarmed attack
- The turn starts with the target already grabbed

For the sake of tidyness we're skipping damage calculations.

In [ ]:
import numpy as np
import xarray

import pathfinder2e_stats as pf2

In [ ]:
# Keen rune?
keen = True

# Fearsome, Greater Fearsome, Crushing, or Greater Crushing Rune?
# What's the status penalty to AC on a critical hit?
fearsome = -2

# Bestial Mutagen or Fury Cocktail for an extra +1 item bonus to attack
# Aid or Albatross Curse for +1 circumstance bonus
# Bless or Corageous Anthem for +1 status bonus
# Heroism for +1 (3rd), +2 (6th), +3 (9th) status bonus
party_setup = {
    "dims": ["party setup"],
    "coords": {"party setup": ["nothing", "everything"]},
}
extra_atk_bonus = xarray.DataArray([0, 5], **party_setup)

# Frightened, sickened, or clumsy when the turn begins?
initial_AC_status_penalty = xarray.DataArray([0, -2], **party_setup)

In [ ]:
common_atk = pf2.tables.PC.level + pf2.tables.PC.attack_item_bonus.potency_rune
martial_atk = (
    common_atk
    + pf2.tables.PC.ability_bonus.boosts.sel(initial=4)
    + pf2.tables.PC.ability_bonus.apex
) + pf2.tables.PC.weapon_proficiency.to_array("class").sel(
    {"class": ["martial", "fighter", "martial", "martial"]}
)
martial_atk.coords["class"] = ["monk", "fighter", "ranger", "martial"]
spellcaster_atk = (
    common_atk + pf2.tables.PC.ability_bonus.boosts.sel(initial=3)
) + pf2.tables.PC.weapon_proficiency.to_array("class").sel({"class": ["spellcaster"]})
atk = xarray.concat([martial_atk, spellcaster_atk], dim="class")
atk = atk.sel(level=20)
atk.to_pandas()

Three standard targets:
- level 18 with low AC
- level 20 with moderate AC
- level 22 with high AC

The target starts grabbed and remains grabbed throughout all strikes.
If a strike fails, the grab is lost and the strikes end.

In [ ]:
AC = pf2.tables.SIMPLE_NPC.AC.sel(level=20) - 2
AC.to_pandas()

In [ ]:
MAP = xarray.DataArray(
    [
        [0, -4, -8],  # Monk
        [0, -3, -6],  # Fighter with Agile Grace
        [0, -3, -6],  # Flurry ranger
        [0, -4, -8],  # Any other martial
        [0, -4, -8],  # Any spellcaster
    ],
    dims=["class", "strike"],
    coords={"class": atk.coords["class"], "strike": [1, 2, 3]},
)
MAP.to_pandas()

In [ ]:
perfected_form = xarray.DataArray(
    [
        [True, False, False, False, False],
        [False, False, False, False, False],
        [False, False, False, False, False],
    ],
    dims=["strike", "class"],
    coords={"strike": [1, 2, 3], "class": atk.coords["class"]},
)
perfected_form.to_pandas()

In [ ]:
kwargs = {
    "bonus": atk + MAP + extra_atk_bonus,
    "perfected_form": perfected_form,
    "keen": keen,
}

In [ ]:
AC_status1 = initial_AC_status_penalty
strike1 = pf2.check(DC=AC + AC_status1, **kwargs).outcome.sel(strike=1, drop=True)

In [ ]:
# Critical hits trigger the fearsome rune
AC_status2 = np.minimum(
    xarray.where(strike1 == pf2.DoS.critical_success, fearsome, 0),
    AC_status1,
)
strike2 = xarray.where(
    strike1 >= pf2.DoS.success,
    pf2.check(DC=AC + AC_status2, **kwargs).outcome.sel(strike=2, drop=True),
    pf2.DoS.no_roll,
)

In [ ]:
# Critical hits trigger the fearsome rune
AC_status3 = np.minimum(
    xarray.where(strike2 == pf2.DoS.critical_success, fearsome, 0),
    AC_status2,
)
strike3 = xarray.where(
    strike2 >= pf2.DoS.success,
    pf2.check(DC=AC + AC_status3, **kwargs).outcome.sel(strike=3, drop=True),
    pf2.DoS.no_roll,
)

In [ ]:
strikes = xarray.concat([strike1, strike2, strike3], dim="strike")
strikes.coords["strike"] = [1, 2, 3]
counts = pf2.outcome_counts(strikes)

## Probability to obtain at least a hit on each strike
A hit on the second strike can only happen if you hit on the first.

A hit on the third strike, and consecutive final slam into the ground, can only happen if you hit on the first two.

#### Extra damage and effects (compared to three iterative strikes)
- 0 hits: 10 falling damage
- 1 hit: 20 falling damage
- 2 hits: 30 falling damage
- 3 hits: 40 falling damage + Crushing Grab + 1 strike + grapple continues into next round

In [ ]:
(
    counts.sel(outcome=["Critical success", "Success"])
    .sum("outcome")
    .stack(row=["challenge", "class"], column=["party setup", "strike"])
    .to_pandas()
    .round(3)
    * 100
)

## Probability to obtain a critical hit on each strike

In [ ]:
(
    counts.sel(outcome="Critical success", drop=True)
    .stack(row=["challenge", "class"], column=["party setup", "strike"])
    .to_pandas()
    .round(3)
    * 100
)